# Bayesian Neural Network Regression

In [ ]:
# Torch BNN installation (Do not run):
import sys
!{sys.executable} -m pip install torchbnn

In [1]:
# MODULE IMPORT
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim

import torchbnn as bnn

In [2]:
# DATA DOWNLOAD
# Download all the required datasets:
%store -r x_train
%store -r x_test
%store -r y_train
%store -r y_test

In [3]:
# DATA PREPARATION
# State the training data type as float values:
x = x_train.float()
y = y_train.float()
# Unsqueeze the data to the appropiate 3D dimension:
x = torch.unsqueeze(x, dim=1)
y = torch.unsqueeze(y, dim=1)

In [4]:
# Display the tensor shape to assess the structure:
print(x.shape)
print(y.shape)

torch.Size([25, 1, 240])
torch.Size([25, 1, 1])


In [5]:
# MODEL ARCHITECTURE
# Set the desired model architecture (Nodes, Layers, Activation Functions):
model = nn.Sequential(
    bnn.BayesLinear(prior_mu=0, prior_sigma=0.1, in_features=240, out_features=48),
    nn.ReLU(),
    bnn.BayesLinear(prior_mu=0, prior_sigma=0.1, in_features=48, out_features=8),
    nn.ReLU(),
    bnn.BayesLinear(prior_mu=0, prior_sigma=0.1, in_features=8, out_features=1),
)

In [6]:
# LOSS FUNCTIONS
# Define the Mean Squared Error loss function:
mse_loss = nn.MSELoss()
# Define the Kullback-Leibler Divergence:
kl_loss = bnn.BKLLoss(reduction='mean', last_layer_only=False)
# Define the optimisation algorithm:
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [7]:
# TRAINING PARAMETERS
# State the weight given to the KL Divergence:
kl_weight = 0.1
# State the number of epochs used for backpropagation training:
epochs = 2000

In [8]:
# BNN TRAINING
# Define the training loop:
for step in range(epochs):
    # Calculate the parameters and associated losses:
    pre = model(x)
    mse = mse_loss(pre, y)
    kl = kl_loss(model)
    cost = mse + kl_weight*kl
    # Optimise the parameters by adjusting the gradient:
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    # Display the training progress for visual feedback:
    if step % 100 == 0 or step == epochs - 1:
        print("Epoch: %5d/%5d, MSE : %2.2f, KL : %2.2f" %
              (step + 1, epochs, mse.item(), kl.item()))
# Display the final MSE and KL loss of the trained BNN:
print('Final Errors:       MSE : %2.2f, KL : %2.2f' % (mse.item(), kl.item()))

Epoch:     1/ 2000, MSE : 1.13, KL : 0.08
Epoch:   101/ 2000, MSE : 0.64, KL : 0.36
Epoch:   201/ 2000, MSE : 0.46, KL : 0.57
Epoch:   301/ 2000, MSE : 0.59, KL : 0.69
Epoch:   401/ 2000, MSE : 0.26, KL : 0.83
Epoch:   501/ 2000, MSE : 0.19, KL : 0.93
Epoch:   601/ 2000, MSE : 0.24, KL : 1.04
Epoch:   701/ 2000, MSE : 0.22, KL : 1.10
Epoch:   801/ 2000, MSE : 0.05, KL : 1.14
Epoch:   901/ 2000, MSE : 0.15, KL : 1.08
Epoch:  1001/ 2000, MSE : 0.06, KL : 1.03
Epoch:  1101/ 2000, MSE : 0.15, KL : 0.95
Epoch:  1201/ 2000, MSE : 0.04, KL : 0.90
Epoch:  1301/ 2000, MSE : 0.10, KL : 0.88
Epoch:  1401/ 2000, MSE : 0.03, KL : 0.85
Epoch:  1501/ 2000, MSE : 0.02, KL : 0.83
Epoch:  1601/ 2000, MSE : 0.03, KL : 0.79
Epoch:  1701/ 2000, MSE : 0.03, KL : 0.80
Epoch:  1801/ 2000, MSE : 0.06, KL : 0.79
Epoch:  1901/ 2000, MSE : 0.04, KL : 0.80
Epoch:  2000/ 2000, MSE : 0.02, KL : 0.74
Final Errors:       MSE : 0.02, KL : 0.74


In [9]:
# DATA PREPARATION
# State the testing data type as float values:
x_test = x_test.float()
# Unsqueeze the data to the appropiate 3D dimension:
x_test = torch.unsqueeze(x_test, dim=1)

In [10]:
print(x_test.shape)

torch.Size([5, 1, 240])


In [11]:
# MODEL OUTPUT
# State the number of desired predictions:
pred = 10
# Create an empty dataframe to contain the predictions:
predictions = pd.DataFrame()
# Loop to make the predictions:
for pred in range(pred):
    # Make a prediction:
    y_predict = model(x_test)
    # Squeeze the prediction back to a 2D matrix:
    y_predict = torch.squeeze(y_predict, dim=1)
    # Convert the tensor to a numpy array:
    np_pred = y_predict.detach().numpy()
    # Convert the numpy array to a dataframe:
    pd_pred = pd.DataFrame(np_pred)
    # Append the prediction to the predictions dataframe:
    predictions[pred] = pd_pred.iloc[:,0]

In [12]:
# Display the dataframe for visual inspection:
predictions

,0,1,2,3,4,5,6,7,8,9
0,-0.100422,0.265459,0.350932,-0.048768,0.368419,-0.125832,0.436770,0.427046,0.176620,0.003099
1,0.267999,0.845866,0.829247,0.371524,0.667011,0.400077,0.753102,0.523391,1.094815,0.504238
2,0.454016,0.812671,0.975830,0.502209,1.136629,0.635282,1.432147,0.867086,1.426483,0.663989
3,-0.182855,-0.065810,-0.432779,-0.357948,-0.086900,-0.355956,-0.077723,-0.084005,-0.090720,-0.234773
4,-0.053049,0.033031,0.093225,-0.143480,0.148916,-0.075994,0.148679,0.119788,0.100543,-0.021568


In [13]:
# PREDICTION DESTANDARDISATION:
# Destandardise the predicted viscosities with the same parameters used for its standardisation:
destand_pred = predictions * 39436.97081 + 301200.0

In [14]:
# Display the dataframe for visual inspection:
destand_pred

,0,1,2,3,4,5,6,7,8,9
0,297239.65625,311668.87500,315039.71875,299276.75000,315729.31250,296237.56250,318424.90625,318041.37500,308165.37500,301322.21875
1,311769.06250,334558.40625,333903.00000,315851.78125,327504.87500,316977.81250,330900.06250,321840.93750,344376.18750,321085.62500
2,319105.00000,333249.28125,339683.78125,321005.59375,346025.21875,326253.59375,357679.53125,335395.25000,357456.15625,327385.71875
3,293988.75000,298604.65625,284132.50000,287083.62500,297772.93750,287162.15625,298134.84375,297887.09375,297622.28125,291941.28125
4,299107.90625,302502.65625,304876.50000,295541.56250,307072.81250,298203.00000,307063.43750,305924.09375,305165.12500,300349.43750


In [16]:
# PREDICTIONS PARAMETERS
# Create empty dataframes to contain the average and standard deviation of the predictions:
performance = pd.DataFrame()
# Calculate the average of the predictions:
performance['Avg'] = destand_pred.iloc[:, :].mean(axis = 1)
# Calculate the standard deviation of the predictions:
performance['Std'] = destand_pred.iloc[:, :].std(axis = 1, ddof = 0)

In [17]:
# Display the dataframes for visual inspection:
performance

,Avg,Std
0,308114.56250,8408.283203
1,325876.81250,9632.580078
2,336323.93750,13155.981445
3,293433.00000,5232.101562
4,302580.65625,3862.135010


In [29]:
# PREDICTION UNCERTAINTY
# Calculate uncertainty percentage in terms of the 95% confidence interval (2 Standard Deviations):
uncertainty = pd.DataFrame()
uncertainty['95% Confidence'] = 2 * performance['Std'] / performance['Avg'] * 100

In [30]:
# Display the dataframe for visual inspection:
uncertainty

,95% Confidence
0,5.457894
1,5.911792
2,7.823399
3,3.566130
4,2.552797


In [32]:
results = pd.DataFrame()
results['Prediction'] = performance['Avg']
results['Uncertainty'] = uncertainty['95% Confidence']

In [33]:
results

,Prediction,Uncertainty
0,308114.56250,5.457894
1,325876.81250,5.911792
2,336323.93750,7.823399
3,293433.00000,3.566130
4,302580.65625,2.552797


In [ ]:
# Author: Pablo Caetano.

# The author is hereby defined as the natural person who adapted the original "software" to serve its designated purpose,
# and who is in turn subject to the copyright notice displayed below.

# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
# SOFTWARE.

# MIT License
# Copyright (c) 2019 Harry Kim